In [11]:
!pip install psycopg2


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [177]:
#import library
import psycopg2

# Create a connection to the database

In [178]:
try:
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres port=5433 password=Guerrero99134")
    print(conn)
except psycopg2.Error as e:
    print('Errror: Could not make connection to the postgres database')
    print(e)

<connection object at 0x000001AD58665140; dsn: 'user=postgres password=xxx dbname=postgres host=127.0.0.1 port=5433', closed: 0>


# Use the connection to get a cursor tha can be used to execute queries

In [179]:
try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print('Error: Could not get curser to the Database')
    print(e)
    

# Set automatic commit to be True so that each action is commited without having to call con.commit() after each command

In [180]:
conn.set_session(autocommit=True)

# Create a database to do the work in

In [184]:
database_name = 'sales_db'
try:
    database_exists_query = f"SELECT 1 FROM pg_database WHERE datname = '{database_name}';"
    result = cur.execute(database_exists_query)
    if not result:
        cur.execute(f"CREATE DATABASE {database_name}")
        print(f"Created database '{database_name}' successfully.")
except psycopg2.Error as e:
    print(e)

la base de datos «sales_db» ya existe



# Add the database name in the connect statement . Let's close our connection to the default database, reconnect to the udacity database, and get a new cursor

In [185]:
try:
    conn.close()
except psycopg2.Errror as e:
    print(e)

try:
    conn = psycopg2.connect("host=127.0.0.1 dbname=sales_db user=postgres port=5433 password=Guerrero99134")
    print(f"Connect to database {database_name}")
except psycopg2.Error as e:
    print('Errror: Could not make connection to the postgres database')
    print(e)

try:
    cur = conn.cursor()
    print(f'Create successful cursor {cur}')
except psycopg2.Error as e:
    print('Error: Could not get curser to the Database')
    print(e)

conn.set_session(autocommit=True)

Connect to database sales_db
Create successful cursor <cursor object at 0x000001AD585FEF80; closed: 0>


# Create tables

In [186]:
try:
    table_territories = """
    CREATE TABLE IF NOT EXISTS territories (
    territory_id INT PRIMARY KEY,
    territory VARCHAR(15) NOT NULL
    )
    """
    table_countries = """
    CREATE TABLE IF NOT EXISTS countries(
    country_id INT PRIMARY KEY,
    country VARCHAR(15) NOT NULL,
    territory_id INT,
    FOREIGN KEY (territory_id)
        REFERENCES territories(territory_id)
        ON DELETE CASCADE ON UPDATE CASCADE
    )
    """ 
    table_cities = """
    CREATE TABLE IF NOT EXISTS cities(
    city_id INT PRIMARY KEY,
    city VARCHAR(15) NOT NULL,
    state VARCHAR(20),
    country_id INT,
    FOREIGN KEY (country_id)
        REFERENCES countries(country_id)
        ON DELETE CASCADE ON UPDATE CASCADE
    )
    """ 
    table_customers = """
    CREATE TABLE IF NOT EXISTS customers(
    customer_id INT PRIMARY KEY,
    customername VARCHAR(50) NOT NULL,
    contactlastname VARCHAR(30),
    contactfirstname VARCHAR(30),
    phone VARCHAR(30),
    addressline1 VARCHAR(60),
    addressline2 VARCHAR(60),
    postalcode VARCHAR(15),
    city_id INT,
    FOREIGN KEY (city_id)
        REFERENCES cities(city_id)
        ON DELETE CASCADE ON UPDATE CASCADE
    )
    """
    table_status = """
    CREATE TABLE IF NOT EXISTS status (
    status_id INT PRIMARY KEY,
    status VARCHAR(15)
    )
    """

    table_orders = """
    CREATE TABLE IF NOT EXISTS orders(
    order_id INT PRIMARY KEY,
    ordernumber INT NOT NULL,
    orderdate DATE,
    qtr_id INT,
    month_id INT,
    year_id INT,
    status_id INT,
    customer_id INT,
    FOREIGN KEY (status_id)
        REFERENCES status(status_id)
        ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (customer_id)
        REFERENCES customers(customer_id)
        ON DELETE CASCADE ON UPDATE CASCADE
    )
    """
    table_products_lines = """
    CREATE TABLE IF NOT EXISTS productlines (
    productline_id INT PRIMARY KEY,
    productline VARCHAR(20)
    )
    """
    table_products = """
    CREATE TABLE IF NOT EXISTS products(
    product_id INT PRIMARY KEY,
    productcode CHAR(9),
    msrp INT,
    productline_id INT,
    FOREIGN KEY (productline_id)
        REFERENCES productlines(productline_id)
        ON DELETE CASCADE ON UPDATE CASCADE
    )
    """ 
    table_dealsizes = """
    CREATE TABLE IF NOT EXISTS dealsizes (
    dealsize_id INT PRIMARY KEY,
    dealsize VARCHAR(10)
    )
    """
    table_orders_products = """
    CREATE TABLE IF NOT EXISTS orders_products(
    order_id INT,
    product_id INT,
    quantityordered INT,
    priceeach FLOAT,
    orderlinenumber INT,
    sales FLOAT,
    dealsize_id INT,
    PRIMARY KEY (order_id,product_id),
    FOREIGN KEY (order_id)
        REFERENCES orders(order_id)
        ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (product_id)
        REFERENCES products(product_id)
        ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (dealsize_id)
        REFERENCES dealsizes(dealsize_id)
        ON DELETE CASCADE ON UPDATE CASCADE
    )
    """
    tables = {'territories':table_territories,'countries':table_countries,'cities':table_cities,
             'customers':table_customers,'status':table_status,'orders':table_orders,'productlines':table_products_lines,
             'products':table_products,'dealsizes':table_dealsizes,'orders_products':table_orders_products}
    for table,query in tables.items(): 
        cur.execute(query)
        print(f"Created table {table} successfully.")
        
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)
    

Created table territories successfully.
Created table countries successfully.
Created table cities successfully.
Created table customers successfully.
Created table status successfully.
Created table orders successfully.
Created table productlines successfully.
Created table products successfully.
Created table dealsizes successfully.
Created table orders_products successfully.


# Upload files

In [187]:
tables_upload = {'territories':'territories','countries':'countries','cities':'citys',
             'customers':'customers','status':'status','orders':'orders','productlines':'product_lines',
             'products':'products','dealsizes':'dealsizes','orders_products':'order_products'}

In [188]:
for table, table_file in tables_upload.items():
    upload_csv = f"""
    COPY {table}
    FROM 'D:/Data Engineer/Portafolio Lorenzo Guerrero/Proyecto ETL/output_data/{table_file}.csv'
    DELIMITER ','
    CSV HEADER
    """
    try:
        cur.execute(upload_csv)
        print(f'Successfully upload table {table}')
    except psycopg2.Error as e:
        # Handle primary key violations by skipping conflicting rows
        if 'llave duplicada viola restricción de unicidad' in str(e):
            print(f'Skipped duplicate rows for table {table}')
        else:
            print("Error: Issue uploading rows")
            print(e)

try:
    conn.close()
    print('Connection close succesfully')
except psycopg2.Errror as e:
    print(e)


Skipped duplicate rows for table territories
Skipped duplicate rows for table countries
Skipped duplicate rows for table cities
Skipped duplicate rows for table customers
Skipped duplicate rows for table status
Skipped duplicate rows for table orders
Skipped duplicate rows for table productlines
Skipped duplicate rows for table products
Skipped duplicate rows for table dealsizes
Skipped duplicate rows for table orders_products
Connection close succesfully
